In [6]:
# -*- coding: utf-8 -*-
"""
Solar Plants: Statistical Analysis, Forecasting (11m train + 1m test),
Evaluation per Location, Cross-Location Comparison, and
External Solar Resource Integration (NASA POWER daily).

This script generates a DataFrame of daily predicted and actual energy values
for the last month of the year (December).

Author: (you)
"""

import os
import re
import math
import gc
import time
from typing import List, Tuple, Dict, Optional

import numpy as np
import pandas as pd
import requests
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from pandas.tseries.offsets import DateOffset
import matplotlib.pyplot as plt

# ==============================
# 0) CONFIG: point to your files
# ==============================
OUTPUT_DIR = "solar_outputs"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Corrected file paths to use relative file names from uploaded data
LOC1_FILES = [
    'Actual_38.55_-75.15_2006_UPV_49MW_5_Min.csv',
    'Actual_38.55_-75.35_2006_UPV_28MW_5_Min.csv',
    'Actual_38.55_-75.45_2006_UPV_49MW_5_Min.csv',
    'Actual_38.55_-75.65_2006_UPV_35MW_5_Min.csv',
    'Actual_38.65_-75.25_2006_UPV_35MW_5_Min.csv',
    'Actual_38.65_-75.65_2006_UPV_70MW_5_Min.csv',
    'Actual_38.75_-75.15_2006_UPV_7MW_5_Min.csv',
    'Actual_38.75_-75.25_2006_UPV_42MW_5_Min.csv',
    'Actual_38.75_-75.65_2006_UPV_14MW_5_Min.csv',
    'Actual_38.85_-75.25_2006_UPV_35MW_5_Min.csv',
]

LOC2_FILES = [
    'Actual_41.75_-70.95_2006_UPV_92MW_5_Min.csv',
    'Actual_41.85_-70.65_2006_UPV_83MW_5_Min.csv',
    'Actual_41.85_-70.85_2006_UPV_18MW_5_Min.csv',
    'Actual_41.85_-70.95_2006_UPV_37MW_5_Min.csv',
    'Actual_41.95_-70.85_2006_UPV_28MW_5_Min.csv',
    'Actual_42.05_-71.15_2006_UPV_18MW_5_Min.csv',
    'Actual_42.15_-70.85_2006_UPV_64MW_5_Min.csv',
    'Actual_42.15_-71.75_2006_UPV_83MW_5_Min.csv',
    'Actual_42.15_-72.25_2006_UPV_28MW_5_Min.csv',
    'Actual_42.25_-71.45_2006_UPV_18MW_5_Min.csv',
]

LOC3_FILES = [
    'Actual_39.15_-74.75_2006_UPV_77MW_5_Min.csv',
    'Actual_39.15_-74.85_2006_UPV_38MW_5_Min.csv',
    'Actual_39.25_-74.65_2006_UPV_102MW_5_Min.csv',
    'Actual_39.25_-74.75_2006_UPV_51MW_5_Min.csv',
    'Actual_39.25_-74.95_2006_UPV_51MW_5_Min.csv',
    'Actual_39.25_-75.05_2006_UPV_51MW_5_Min.csv',
    'Actual_39.25_-75.15_2006_UPV_77MW_5_Min.csv',
    'Actual_39.35_-74.65_2006_UPV_115MW_5_Min.csv',
    'Actual_39.35_-74.95_2006_UPV_51MW_5_Min.csv',
    'Actual_39.35_-75.15_2006_UPV_90MW_5_Min.csv',
]

# =================================
# 1) Helpers: parsing & normalization
# =================================
FILENAME_RE = re.compile(
    r"Actual_(-?\d+\.\d+)_(-?\d+\.\d+)_(\d+)_UPV_(\d+)MW_5_Min\.csv",
    re.IGNORECASE
)

CANDIDATE_TIME_FORMATS = [
    "%d-%m-%Y %H:%M",
    "%Y-%m-%d %H:%M",
    "%m/%d/%y %H:%M",
    "%d/%m/%y %H:%M",
    "%m/%d/%Y %H:%M",
    "%d/%m/%Y %H:%M",
]

def robust_parse_datetime(series: pd.Series) -> pd.Series:
    """Parse a timestamp Series using several common formats, then fallback."""
    s = series.astype(str).str.strip()
    parsed = pd.Series(pd.NaT, index=s.index, dtype="datetime64[ns]")
    remaining = s.notna()
    for fmt in CANDIDATE_TIME_FORMATS:
        try:
            tmp = pd.to_datetime(s.where(remaining), format=fmt, errors="coerce")
            fill = remaining & tmp.notna()
            parsed.loc[fill] = tmp.loc[fill]
            remaining = remaining & parsed.isna()
            if not remaining.any():
                break
        except Exception:
            pass
    if parsed.isna().any():
        tmp = pd.to_datetime(s.where(parsed.isna()), errors="coerce", dayfirst=False)
        parsed = parsed.where(parsed.notna(), tmp)
    return parsed

def find_col(cols: List[str], candidates: List[str]) -> Optional[str]:
    """Return the first existing column (case-insensitive, strip/BOM cleaned) among candidates."""
    norm = {c.strip().replace("\ufeff", "").lower(): c for c in cols}
    for cand in candidates:
        key = cand.strip().lower()
        if key in norm:
            return norm[key]
    return None

def load_location(files: List[str], location_name: str) -> pd.DataFrame:
    """
    Load & clean one location's CSVs.
    """
    if not files:
        raise ValueError(f"[{location_name}] No files specified.")
    frames = []
    for path in files:
        # Use relative file path from the /content/ directory
        file_path = os.path.join('/content/', path)
        if not os.path.exists(file_path):
            print(f"[{location_name}] Skipping missing file: {file_path}")
            continue
        df = pd.read_csv(file_path)
        df.columns = [c.strip().replace("\ufeff", "") for c in df.columns]
        time_col = find_col(df.columns, ["LocalTime", "Timestamp", "DateTime", "Time"])
        power_col = find_col(df.columns, ["Power(MW)", "Power", "PowerMW", "AC Power", "ActivePower(MW)"])
        if time_col is None or power_col is None:
            raise KeyError(f"[{location_name}] Missing required columns in {os.path.basename(path)}.")
        df = df[[time_col, power_col]].rename(columns={time_col: "Timestamp", power_col: "Power"})
        df["Timestamp"] = robust_parse_datetime(df["Timestamp"])
        df["Power"] = pd.to_numeric(df["Power"], errors="coerce")
        m = FILENAME_RE.search(os.path.basename(path))
        if not m:
            raise ValueError(f"[{location_name}] Filename does not match expected pattern: {os.path.basename(path)}")
        lat, lon, year, cap = float(m.group(1)), float(m.group(2)), int(m.group(3)), int(m.group(4))
        df["Latitude"] = lat
        df["Longitude"] = lon
        df["Year"] = year
        df["Capacity(MW)"] = cap
        df["PlantID"] = f"{lat}_{lon}"
        df = df.dropna(subset=["Timestamp", "Power"])
        df = df[df["Power"] >= 0]
        frames.append(df)
    if not frames:
        raise ValueError(f"[{location_name}] No valid CSVs loaded after cleaning.")
    out = pd.concat(frames, ignore_index=True)
    out = out.loc[:, ~out.columns.duplicated()].copy()
    out = out.sort_values("Timestamp").reset_index(drop=True)
    out["LocationName"] = location_name
    out["EnergyMWh"] = out["Power"] * (5.0 / 60.0)
    return out

# =========================================
# 2) NASA POWER: daily irradiance & weather
# =========================================
NASA_BASE = "https://power.larc.nasa.gov/api/temporal/daily/point"
NASA_PARAMS = [
    "ALLSKY_SFC_SW_DWN",    # GHI (kWh/m^2/day)
    "CLRSKY_SFC_SW_DWN",    # Clear-sky GHI (kWh/m^2/day)
    "CLOUD_AMT",            # Cloud amount (%)
    "T2M",                  # 2m Air temperature (°C)
    "WS10M",                # 10m Wind speed (m/s)
    "RH2M",                 # 2m Relative humidity (%)
    "PRECTOTCORR"           # Precipitation (mm/day)
]

def nasa_cache_path(lat: float, lon: float, year: int) -> str:
    safe = f"{lat:.2f}_{lon:.2f}_{year}"
    return os.path.join(OUTPUT_DIR, f"nasa_power_daily_{safe}.csv")

def fetch_nasa_power_daily(lat: float, lon: float, year: int, retry: int = 2, wait_s: float = 1.5) -> pd.DataFrame:
    """
    Fetch NASA POWER daily data for one point/year.
    Caches to CSV; on failure returns empty DataFrame.
    """
    cache = nasa_cache_path(lat, lon, year)
    if os.path.exists(cache):
        try:
            df = pd.read_csv(cache, parse_dates=["Date"])
            return df
        except Exception:
            pass
    start = f"{year}0101"
    end   = f"{year}1231"
    params = {
        "latitude": lat,
        "longitude": lon,
        "start": start,
        "end": end,
        "parameters": ",".join(NASA_PARAMS),
        "community": "RE",
        "format": "JSON"
    }
    last_err = None
    for _ in range(retry + 1):
        try:
            r = requests.get(NASA_BASE, params=params, timeout=30)
            r.raise_for_status()
            data = r.json()
            param_block = data.get("properties", {}).get("parameter", {})
            if not param_block:
                raise ValueError("NASA POWER: empty 'parameter' block")
            df = pd.DataFrame(index=pd.to_datetime(sorted(next(iter(param_block.values())).keys())))
            for k, d in param_block.items():
                s = pd.Series(d, dtype="float64")
                s.index = pd.to_datetime(s.index)
                df[k] = s
            df = df.reset_index().rename(columns={"index": "Date"})
            df.to_csv(cache, index=False)
            return df
        except Exception as e:
            last_err = e
            time.sleep(wait_s)
    print(f" NASA POWER fetch failed for ({lat}, {lon}, {year}): {last_err}")
    return pd.DataFrame(columns=["Date"] + NASA_PARAMS)

def daily_energy_by_plant(df_loc: pd.DataFrame) -> pd.DataFrame:
    """
    Sum 5-min energy to daily per plant; compute daily capacity factor.
    Returns columns: Date, PlantID, Capacity_MW, DailyEnergyMWh, DailyCF
    """
    d = df_loc.copy()
    d["Date"] = d["Timestamp"].dt.floor("D")
    agg = (d.groupby(["PlantID", "Date"])
              .agg(DailyEnergyMWh=("EnergyMWh", "sum"),
                   Capacity_MW=("Capacity(MW)", "max"))
              .reset_index())
    agg["DailyCF"] = agg["DailyEnergyMWh"] / (agg["Capacity_MW"] * 24.0)
    return agg

def enrich_with_weather(df_loc: pd.DataFrame, location_name: str) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    For each plant (lat,lon,year), fetch NASA POWER daily weather and
    merge with plant daily energy/CF. Returns:
    - merged_daily: per-plant daily energy + weather
    - plant_weather_summary: per-plant means and CF↔GHI corr merged in)
    """
    daily = daily_energy_by_plant(df_loc)
    plants = (df_loc.groupby("PlantID")
                     .agg(Latitude=("Latitude", "first"),
                          Longitude=("Longitude", "first"),
                          Year=("Year", "min"))
                     .reset_index())

    merged_blocks = []
    summaries = []
    for _, row in plants.iterrows():
        pid = row["PlantID"]; lat = float(row["Latitude"]); lon = float(row["Longitude"]); year = int(row["Year"])
        wx = fetch_nasa_power_daily(lat, lon, year)
        if wx.empty:
            join = daily[daily["PlantID"] == pid].merge(pd.DataFrame({"Date": pd.to_datetime([])}), on="Date", how="left")
        else:
            join = daily[daily["PlantID"] == pid].merge(wx, on="Date", how="left")
        join["PlantID"] = pid
        merged_blocks.append(join)
        s = {"PlantID": pid,
             "MeanGHI_kWhm2_day": join["ALLSKY_SFC_SW_DWN"].mean(skipna=True),
             "MeanClearSkyGHI_kWhm2_day": join["CLRSKY_SFC_SW_DWN"].mean(skipna=True),
             "MeanCloudAmt_pct": join["CLOUD_AMT"].mean(skipna=True),
             "MeanT2M_C": join["T2M"].mean(skipna=True),
             "MeanWS10M_ms": join["WS10M"].mean(skipna=True),
             "MeanRH2M_pct": join["RH2M"].mean(skipna=True),
             "MeanPrecip_mm_day": join["PRECTOTCORR"].mean(skipna=True),
        }
        cf = join["DailyCF"]
        ghi = join["ALLSKY_SFC_SW_DWN"]
        s["Corr_CF_vs_GHI"] = float(cf.corr(ghi)) if (cf.notna().sum() > 5 and ghi.notna().sum() > 5) else np.nan
        summaries.append(s)
    merged_daily = pd.concat(merged_blocks, ignore_index=True)
    merged_daily["LocationName"] = location_name
    plant_weather_summary = pd.DataFrame(summaries)
    plant_weather_summary["LocationName"] = location_name
    merged_daily.to_csv(os.path.join(OUTPUT_DIR, f"{location_name.replace(' ', '')}_weather_daily.csv"), index=False)
    return merged_daily, plant_weather_summary

# =========================================
# 3) Stats, Forecasting, and Evaluations
# =========================================
def plant_statistics(df_loc: pd.DataFrame) -> pd.DataFrame:
    """Compute plant-level stats and capacity factor."""
    stats = (df_loc.groupby("PlantID")["Power"].agg(mean="mean", median="median", std="std", max="max").reset_index())
    energy = (df_loc.groupby("PlantID").agg(TotalEnergyMWh=("EnergyMWh", "sum"), Intervals=("EnergyMWh", "count")).reset_index())
    meta = (df_loc.groupby("PlantID").agg(Capacity_MW=("Capacity(MW)", "max"), Latitude=("Latitude", "first"), Longitude=("Longitude", "first"), Year=("Year", "min")).reset_index())
    out = stats.merge(energy, on="PlantID").merge(meta, on="PlantID")
    total_hours = out["Intervals"] * (5.0 / 60.0)
    out["CapacityFactor"] = out["TotalEnergyMWh"] / (out["Capacity_MW"] * total_hours).replace(0, np.nan)
    return out

def df_split_by_month(df: pd.DataFrame) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """Time-based split: November train, December test."""
    train = df[df["Timestamp"].dt.month == 11].copy()
    test = df[df["Timestamp"].dt.month == 12].copy()
    return train, test


def forecast_one_plant(df_plant: pd.DataFrame, location_name: str) -> Dict[str, any]:
    """
    Prophet forecast with 11/1 split; returns MAE/RMSE/R2/MAPE, and #test points,
    and also the y_true and y_pred values.
    """
    df_plant = df_plant.dropna(subset=["Timestamp", "Power"])
    if df_plant.empty:
        return {"MAE": np.nan, "RMSE": np.nan, "R2": np.nan, "MAPE": np.nan, "N_test": 0, "y_true": [], "y_pred": []}

    # Use the new df_split_by_month function
    train, test = df_split_by_month(df_plant)
    if train.empty or test.empty:
        return {"MAE": np.nan, "RMSE": np.nan, "R2": np.nan, "MAPE": np.nan, "N_test": 0, "y_true": [], "y_pred": []}

    tr = train.rename(columns={"Timestamp": "ds", "Power": "y"})

    # Fine-tuning: Use a flexible changepoint_prior_scale for locations 1 and 2
    if location_name in ["Location 1", "Location 2"]:
        model = Prophet(daily_seasonality=True, weekly_seasonality=True, yearly_seasonality=True, changepoint_prior_scale=0.08)
    else:
        # Use default for Location 3 as per request
        model = Prophet(daily_seasonality=True, weekly_seasonality=True, yearly_seasonality=True)

    model.fit(tr)

    future = pd.DataFrame({"ds": test["Timestamp"].values})
    fcst = model.predict(future)

    y_true = test["Power"].to_numpy()
    y_pred = np.maximum(0, fcst["yhat"].to_numpy())

    mae = mean_absolute_error(y_true, y_pred)
    rmse = math.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)

    # Filter out zero actuals to avoid division by zero
    valid_indices = y_true > 0
    mape = np.nan
    if np.any(valid_indices):
        mape = np.mean(np.abs((y_true[valid_indices] - y_pred[valid_indices]) / y_true[valid_indices])) * 100

    return {"MAE": mae, "RMSE": rmse, "R2": r2, "MAPE": mape, "N_test": len(test), "y_true": y_true, "y_pred": y_pred}

def evaluate_location(df_loc: pd.DataFrame, location_name: str) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """
    - Returns (plant_stats_df, plant_metrics_df, daily_forecast_df)
    """
    stats_df = plant_statistics(df_loc)
    _merged_daily, wx_summary = enrich_with_weather(df_loc, location_name)
    stats_df = stats_df.merge(wx_summary, on="PlantID", how="left")
    metrics_rows = []

    all_predicted = pd.DataFrame()
    all_actual = pd.DataFrame()

    for pid, grp in df_loc.groupby("PlantID"):
        res = forecast_one_plant(grp[["Timestamp", "Power"]].copy(), location_name)

        if res["N_test"] > 0:
            predicted_df = pd.DataFrame({'Predicted_Power': res['y_pred']}, index=grp["Timestamp"][-len(res["y_pred"]):])
            actual_df = pd.DataFrame({'Actual_Power': res['y_true']}, index=grp["Timestamp"][-len(res["y_true"]):])

            if all_predicted.empty:
                all_predicted = predicted_df
            else:
                all_predicted = all_predicted.add(predicted_df, fill_value=0)

            if all_actual.empty:
                all_actual = actual_df
            else:
                all_actual = all_actual.add(actual_df, fill_value=0)

        metrics_rows.append({"PlantID": pid, "MAE": res['MAE'], "RMSE": res['RMSE'], "R2": res['R2'], "MAPE": res['MAPE'], "N_test": res['N_test']})
        gc.collect()

    daily_predicted = all_predicted.resample('D').sum().rename(columns={'Predicted_Power': 'Predicted Daily Energy'})
    daily_actual = all_actual.resample('D').sum().rename(columns={'Actual_Power': 'Actual Daily Energy'})

    daily_forecast_df = pd.merge(daily_actual, daily_predicted, left_index=True, right_index=True, how='outer')
    daily_forecast_df.index.name = 'Date'

    metrics_df = pd.DataFrame(metrics_rows)
    stats_df["LocationName"] = location_name
    metrics_df["LocationName"] = location_name
    return stats_df, metrics_df, daily_forecast_df

def compare_locations(stats_all: pd.DataFrame, metrics_all: pd.DataFrame) -> pd.DataFrame:
    """Aggregate to location level for comparison, including AvgGHI."""
    energy_loc = (stats_all.groupby("LocationName")["TotalEnergyMWh"].sum().rename("TotalEnergyMWh"))
    cf_loc = (stats_all.groupby("LocationName")["CapacityFactor"].mean().rename("AvgPlantCapacityFactor"))

    err_loc = (metrics_all.groupby("LocationName")[["MAE", "RMSE", "R2", "MAPE"]].mean().rename(columns={"MAE": "AvgMAE", "RMSE": "AvgRMSE", "R2": "AvgR2", "MAPE": "AvgMAPE"}))

    ctx = (stats_all.groupby("LocationName").agg(NumPlants=("PlantID", "nunique"), MaxPlantCapacityMW=("Capacity_MW", "max"), MedianLatitude=("Latitude", "median")))
    ghi_loc = (stats_all.groupby("LocationName")["MeanGHI_kWhm2_day"].mean().rename("AvgGHI_kWhm2_day"))
    comp = pd.concat([energy_loc, cf_loc, err_loc, ctx, ghi_loc], axis=1).reset_index()
    return comp.sort_values("TotalEnergyMWh", ascending=False)

# =====================
# 4) Run end-to-end
# =====================
all_stats = []
all_metrics = []
all_forecasts = {}

def process_location(files: List[str], name: str):
    df = load_location(files, name)
    stats, metrics, daily_forecast_df = evaluate_location(df, name)
    stats.to_csv(os.path.join(OUTPUT_DIR, f"{name.replace(' ', '')}_stats.csv"), index=False)
    metrics.to_csv(os.path.join(OUTPUT_DIR, f"{name.replace(' ', '')}_metrics.csv"), index=False)
    print(f"✔ {name}: {stats['PlantID'].nunique()} plants, "
          f"{len(df):,} rows, energy={stats['TotalEnergyMWh'].sum():,.1f} MWh")
    return stats, metrics, daily_forecast_df

if LOC1_FILES:
    s1, m1, f1 = process_location(LOC1_FILES, "Location 1")
    all_stats.append(s1); all_metrics.append(m1); all_forecasts['Location 1'] = f1
if LOC2_FILES:
    s2, m2, f2 = process_location(LOC2_FILES, "Location 2")
    all_stats.append(s2); all_metrics.append(m2); all_forecasts['Location 2'] = f2
if LOC3_FILES:
    s3, m3, f3 = process_location(LOC3_FILES, "Location 3")
    all_stats.append(s3); all_metrics.append(m3); all_forecasts['Location 3'] = f3

if not all_stats or not all_metrics:
    raise SystemExit("No locations processed. Fill LOC1_FILES / LOC2_FILES / LOC3_FILES and re-run.")

stats_all = pd.concat(all_stats, ignore_index=True)
metrics_all = pd.concat(all_metrics, ignore_index=True)

comp = compare_locations(stats_all, metrics_all)
comp.to_csv(os.path.join(OUTPUT_DIR, "CrossLocation_Comparison.csv"), index=False)

# Print final daily forecast DataFrame
for name, df_forecast in all_forecasts.items():
    print(f"\n===== Daily Predicted vs. Actual Energy for {name} =====")
    print(df_forecast.to_string())

DEBUG:cmdstanpy:input tempfile: /tmp/tmp6la3fd5l/7njpizxz.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp6la3fd5l/0tzaoxnx.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.12/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=81667', 'data', 'file=/tmp/tmp6la3fd5l/7njpizxz.json', 'init=/tmp/tmp6la3fd5l/0tzaoxnx.json', 'output', 'file=/tmp/tmp6la3fd5l/prophet_modeluoy_icon/prophet_model-20250919055109.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
05:51:09 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
05:51:11 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
DEBUG:cmdstanpy:input tempfile: /tmp/tmp6la3fd5l/3ajzzyqt.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp6la3fd5l/f1oj6nin.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/

✔ Location 1: 10 plants, 1,051,200 rows, energy=501,179.4 MWh


DEBUG:cmdstanpy:input tempfile: /tmp/tmp6la3fd5l/iow4barf.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp6la3fd5l/11gz_wpm.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.12/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=22339', 'data', 'file=/tmp/tmp6la3fd5l/iow4barf.json', 'init=/tmp/tmp6la3fd5l/11gz_wpm.json', 'output', 'file=/tmp/tmp6la3fd5l/prophet_modelaa4i1mma/prophet_model-20250919055149.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
05:51:49 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
05:51:50 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
DEBUG:cmdstanpy:input tempfile: /tmp/tmp6la3fd5l/i7mya571.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp6la3fd5l/0vvaad48.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/

✔ Location 2: 10 plants, 1,051,200 rows, energy=606,467.8 MWh


DEBUG:cmdstanpy:input tempfile: /tmp/tmp6la3fd5l/a010vold.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp6la3fd5l/lhr69e7q.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.12/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=98605', 'data', 'file=/tmp/tmp6la3fd5l/a010vold.json', 'init=/tmp/tmp6la3fd5l/lhr69e7q.json', 'output', 'file=/tmp/tmp6la3fd5l/prophet_model3y8vmscs/prophet_model-20250919055231.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
05:52:31 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
05:52:32 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
DEBUG:cmdstanpy:input tempfile: /tmp/tmp6la3fd5l/l_m80uzi.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp6la3fd5l/76593gv_.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/

✔ Location 3: 10 plants, 1,051,200 rows, energy=949,706.9 MWh

===== Daily Predicted vs. Actual Energy for Location 1 =====
            Actual Daily Energy  Predicted Daily Energy
Date                                                   
2006-12-01              4598.75            19924.356965
2006-12-02             14364.56            20626.344869
2006-12-03              6507.14            16161.334763
2006-12-04             16306.01            13514.478202
2006-12-05             14269.44            12393.242340
2006-12-06             14580.67             9519.655821
2006-12-07             13781.20             9751.355848
2006-12-08             14561.24            13455.518524
2006-12-09             12603.82            12074.797562
2006-12-10             15802.40             7960.539255
2006-12-11             14223.94             5725.778798
2006-12-12             14484.07             4720.560896
2006-12-13              2906.16             3374.595107
2006-12-14              7292.10     

In [9]:
print(comp)

  LocationName  TotalEnergyMWh  AvgPlantCapacityFactor     AvgMAE    AvgRMSE  \
2   Location 3   949706.916667                0.154692   7.244106  13.598898   
1   Location 2   606467.791667                0.145362  51.259663  60.742093   
0   Location 1   501179.406667                0.158770   3.925417   6.790876   

        AvgR2      AvgMAPE  NumPlants  MaxPlantCapacityMW  MedianLatitude  \
2    0.044201   309.262821         10                 115           39.25   
1 -116.070731  4965.663228         10                  92           42.00   
0   -0.152553   236.848156         10                  70           38.65   

   AvgGHI_kWhm2_day  
2          4.162727  
1          3.763252  
0          4.269275  


In [10]:
highest_energy_location = comp.sort_values(by='TotalEnergyMWh', ascending=False).iloc[0]
print("\nLocation with the highest energy generation:")
print(highest_energy_location[['LocationName', 'TotalEnergyMWh']])


Location with the highest energy generation:
LocationName         Location 3
TotalEnergyMWh    949706.916667
Name: 2, dtype: object


In [11]:
print("Cross-Location Comparison (comp):")
display(comp)

print("\nPlant-level Statistics (stats_all):")
display(stats_all)

Cross-Location Comparison (comp):


,LocationName,TotalEnergyMWh,AvgPlantCapacityFactor,AvgMAE,AvgRMSE,AvgR2,AvgMAPE,NumPlants,MaxPlantCapacityMW,MedianLatitude,AvgGHI_kWhm2_day
2,Location 3,949706.916667,0.154692,7.244106,13.598898,0.044201,309.262821,10,115,39.25,4.162727
1,Location 2,606467.791667,0.145362,51.259663,60.742093,-116.070731,4965.663228,10,92,42.00,3.763252
0,Location 1,501179.406667,0.158770,3.925417,6.790876,-0.152553,236.848156,10,70,38.65,4.269275



Plant-level Statistics (stats_all):


,PlantID,mean,median,std,max,TotalEnergyMWh,Intervals,Capacity_MW,Latitude,Longitude,...,CapacityFactor,MeanGHI_kWhm2_day,MeanClearSkyGHI_kWhm2_day,MeanCloudAmt_pct,MeanT2M_C,MeanWS10M_ms,MeanRH2M_pct,MeanPrecip_mm_day,Corr_CF_vs_GHI,LocationName
0,38.55_-75.15,7.404969,0.0,11.375085,46.50,64867.525000,105120,49,38.55,-75.15,...,0.151122,4.269275,5.474506,52.735589,14.441781,5.551616,79.924658,3.297863,0.883446,Location 1
1,38.55_-75.35,4.200801,0.0,6.445523,24.70,36799.016667,105120,28,38.55,-75.35,...,0.150029,4.269275,5.474506,52.735589,14.397671,4.517068,76.781699,2.810685,0.889827,Location 1
2,38.55_-75.45,7.323873,0.0,11.209178,45.70,64157.125000,105120,49,38.55,-75.45,...,0.149467,4.269275,5.474506,52.735589,14.397671,4.517068,76.781699,2.810685,0.899213,Location 1
3,38.55_-75.65,6.010917,0.0,8.391357,29.80,52655.633333,105120,35,38.55,-75.65,...,0.171740,4.269275,5.474506,52.735589,14.397671,4.517068,76.781699,2.810685,0.968522,Location 1
4,38.65_-75.25,6.007893,0.0,8.419685,29.50,52629.141667,105120,35,38.65,-75.25,...,0.171654,4.269275,5.474506,52.735589,14.441781,5.551616,79.924658,3.297863,0.958803,Location 1
5,38.65_-75.65,10.568103,0.0,16.093857,63.70,92576.583333,105120,70,38.65,-75.65,...,0.150973,4.269275,5.474506,52.735589,14.397671,4.517068,76.781699,2.810685,0.900887,Location 1
6,38.75_-75.15,1.210248,0.0,1.685166,5.85,10601.773333,105120,7,38.75,-75.15,...,0.172893,4.269275,5.474506,52.735589,14.069836,5.475507,79.154712,2.907397,0.954543,Location 1
7,38.75_-75.25,7.227961,0.0,10.094314,37.20,63316.941667,105120,42,38.75,-75.25,...,0.172094,4.269275,5.474506,52.735589,14.069836,5.475507,79.154712,2.907397,0.960122,Location 1
8,38.75_-75.65,2.108696,0.0,3.227891,12.80,18472.175000,105120,14,38.75,-75.65,...,0.150621,4.269275,5.474506,52.735589,13.992658,4.403973,76.598575,2.682438,0.898871,Location 1
9,38.85_-75.25,5.148800,0.0,7.902286,31.00,45103.491667,105120,35,38.85,-75.25,...,0.147109,4.269275,5.474506,52.735589,14.069836,5.475507,79.154712,2.907397,0.891607,Location 1


In [12]:
print("Cross-Location Comparison:")
display(comp)

highest_energy_location = comp.sort_values(by='TotalEnergyMWh', ascending=False).iloc[0]
print("\nLocation with the highest energy generation:")
display(highest_energy_location[['LocationName', 'TotalEnergyMWh']].to_frame().T)

print("\nFactors Influencing Energy Generation:")
print(f"- Location 3 has the highest total energy generation ({highest_energy_location['TotalEnergyMWh']:.2f} MWh).")
print(f"- Based on the 'comp' DataFrame:")
print(f"  - Location 1 has the highest Average GHI ({comp[comp['LocationName'] == 'Location 1']['AvgGHI_kWhm2_day'].iloc[0]:.2f} kWh/m2/day), but not the highest total energy.")
print(f"  - Location 3 has a lower Average GHI ({highest_energy_location['AvgGHI_kWhm2_day']:.2f} kWh/m2/day) but the highest total energy, likely due to a higher total installed capacity (indicated by MaxPlantCapacityMW and the sum of individual plant capacities in stats_all).")
print(f"  - Location 2 has the lowest Average GHI ({comp[comp['LocationName'] == 'Location 2']['AvgGHI_kWhm2_day'].iloc[0]:.2f} kWh/m2/day) and moderate total energy.")
print(f"- The 'stats_all' DataFrame provides detailed plant-level statistics (including individual plant capacity and average GHI) that were used to support these observations and recommendations.")

print("\nRecommendations for Improving Energy Generation:")
print("- Location 1: Focus on optimizing existing plant performance and leveraging favorable GHI with advanced technologies.")
print("- Location 2: Consider strategic capacity expansion due to a good balance of GHI and energy generation per plant.")
print("- Location 3: Address potential performance issues in individual plants and explore alternative technologies due to lower GHI.")

print("\nDetailed Plant-Level Statistics (stats_all):")
display(stats_all)

Cross-Location Comparison:


,LocationName,TotalEnergyMWh,AvgPlantCapacityFactor,AvgMAE,AvgRMSE,AvgR2,AvgMAPE,NumPlants,MaxPlantCapacityMW,MedianLatitude,AvgGHI_kWhm2_day
2,Location 3,949706.916667,0.154692,7.244106,13.598898,0.044201,309.262821,10,115,39.25,4.162727
1,Location 2,606467.791667,0.145362,51.259663,60.742093,-116.070731,4965.663228,10,92,42.00,3.763252
0,Location 1,501179.406667,0.158770,3.925417,6.790876,-0.152553,236.848156,10,70,38.65,4.269275



Location with the highest energy generation:


,LocationName,TotalEnergyMWh
2,Location 3,949706.916667



Factors Influencing Energy Generation:
- Location 3 has the highest total energy generation (949706.92 MWh).
- Based on the 'comp' DataFrame:
  - Location 1 has the highest Average GHI (4.27 kWh/m2/day), but not the highest total energy.
  - Location 3 has a lower Average GHI (4.16 kWh/m2/day) but the highest total energy, likely due to a higher total installed capacity (indicated by MaxPlantCapacityMW and the sum of individual plant capacities in stats_all).
  - Location 2 has the lowest Average GHI (3.76 kWh/m2/day) and moderate total energy.
- The 'stats_all' DataFrame provides detailed plant-level statistics (including individual plant capacity and average GHI) that were used to support these observations and recommendations.

Recommendations for Improving Energy Generation:
- Location 1: Focus on optimizing existing plant performance and leveraging favorable GHI with advanced technologies.
- Location 2: Consider strategic capacity expansion due to a good balance of GHI and energy

,PlantID,mean,median,std,max,TotalEnergyMWh,Intervals,Capacity_MW,Latitude,Longitude,...,CapacityFactor,MeanGHI_kWhm2_day,MeanClearSkyGHI_kWhm2_day,MeanCloudAmt_pct,MeanT2M_C,MeanWS10M_ms,MeanRH2M_pct,MeanPrecip_mm_day,Corr_CF_vs_GHI,LocationName
0,38.55_-75.15,7.404969,0.0,11.375085,46.50,64867.525000,105120,49,38.55,-75.15,...,0.151122,4.269275,5.474506,52.735589,14.441781,5.551616,79.924658,3.297863,0.883446,Location 1
1,38.55_-75.35,4.200801,0.0,6.445523,24.70,36799.016667,105120,28,38.55,-75.35,...,0.150029,4.269275,5.474506,52.735589,14.397671,4.517068,76.781699,2.810685,0.889827,Location 1
2,38.55_-75.45,7.323873,0.0,11.209178,45.70,64157.125000,105120,49,38.55,-75.45,...,0.149467,4.269275,5.474506,52.735589,14.397671,4.517068,76.781699,2.810685,0.899213,Location 1
3,38.55_-75.65,6.010917,0.0,8.391357,29.80,52655.633333,105120,35,38.55,-75.65,...,0.171740,4.269275,5.474506,52.735589,14.397671,4.517068,76.781699,2.810685,0.968522,Location 1
4,38.65_-75.25,6.007893,0.0,8.419685,29.50,52629.141667,105120,35,38.65,-75.25,...,0.171654,4.269275,5.474506,52.735589,14.441781,5.551616,79.924658,3.297863,0.958803,Location 1
5,38.65_-75.65,10.568103,0.0,16.093857,63.70,92576.583333,105120,70,38.65,-75.65,...,0.150973,4.269275,5.474506,52.735589,14.397671,4.517068,76.781699,2.810685,0.900887,Location 1
6,38.75_-75.15,1.210248,0.0,1.685166,5.85,10601.773333,105120,7,38.75,-75.15,...,0.172893,4.269275,5.474506,52.735589,14.069836,5.475507,79.154712,2.907397,0.954543,Location 1
7,38.75_-75.25,7.227961,0.0,10.094314,37.20,63316.941667,105120,42,38.75,-75.25,...,0.172094,4.269275,5.474506,52.735589,14.069836,5.475507,79.154712,2.907397,0.960122,Location 1
8,38.75_-75.65,2.108696,0.0,3.227891,12.80,18472.175000,105120,14,38.75,-75.65,...,0.150621,4.269275,5.474506,52.735589,13.992658,4.403973,76.598575,2.682438,0.898871,Location 1
9,38.85_-75.25,5.148800,0.0,7.902286,31.00,45103.491667,105120,35,38.85,-75.25,...,0.147109,4.269275,5.474506,52.735589,14.069836,5.475507,79.154712,2.907397,0.891607,Location 1


In [15]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
